<a href="https://colab.research.google.com/github/Gioalf/AH2179/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Bus arrival delays regression modeling**


In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/zhenliangma/Applied-AI-in-Transportation/master/Exercise_2_regression_model/Exercise2BusData.csv'
df = pd.read_csv(url)

# df = pd.read_csv('Exercise2BusData.csv')
df.head(10)

,Arrival_time,Stop_id,Bus_id,Line_id,Arrival_delay,Dwell_time,Scheduled_travel_time,Upstream_stop_delay,Recurrent_delay
0,01/05/2022 07:03,10033,41355,1,-23,0,39,-4,28.641509
1,01/05/2022 07:21,10033,41356,1,168,0,39,181,27.880597
2,01/05/2022 07:37,10033,44400,1,231,0,39,244,27.852941
3,01/05/2022 07:50,10033,44399,1,150,17,39,143,27.826087
4,01/05/2022 19:29,10033,44060,1,965,35,51,947,42.029412
5,01/05/2022 19:58,10033,41351,1,33,19,46,30,38.970588
6,01/05/2022 20:14,10033,44399,1,75,26,46,63,38.840580
7,01/05/2022 20:28,10033,41356,1,58,15,46,52,34.059701
8,01/05/2022 20:57,10033,44060,1,904,35,46,888,34.102941
9,01/05/2022 21:00,10033,41357,1,141,12,46,140,34.000000


Let's try to check different approaches, one that exploits the large database we have with all the 2179 entries, and others that only uses a limited number of random entries to check how large of a difference we get.


Remove columns that are no longer needed.

In [2]:
df = df.drop(['Arrival_time','Stop_id','Bus_id','Line_id'], axis=1)

I'll write a short loop that runs the linear regression with the different number of rows we select.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

metrics = pd.DataFrame(columns=['Number of rows', 'MAE', 'MSE', 'R2'])

for i, num in enumerate([50, 200, 500, 2179]): #select the different number entries to evaluate
  df_sample = df.sample(n=num, random_state=42) #creating the sample df with the random entries

  x = df_sample.drop(['Arrival_delay'], axis=1) # selecting the independent variables
  y = df_sample['Arrival_delay'] #selecting the dependent variable
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) #train and test pools

  # Create a Linear Regression model
  model = LinearRegression()
  model.fit(X_train, y_train)# Fit the model to the training data
  y_pred = model.predict(X_test)# Predict the test data with the fitted model

  #evaluation of the metrics
  mae = mean_absolute_error(y_test, y_pred)
  mse = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  #Storing the values
  metrics.loc[i]=[num,mae,mse,r2]

In [9]:
metrics

,Number of rows,MAE,MSE,R2
0,50.0,15.925672,526.951283,0.968390
1,200.0,9.852195,171.089590,0.990398
2,500.0,10.409570,208.023546,0.995662
3,2179.0,9.339559,150.404587,0.996024
